## 2. Semantic Search - ingest/search

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    SearchField,  
    VectorSearch,
    SemanticSearch,
    SemanticPrioritizedFields,
    SemanticField,
    HnswAlgorithmConfiguration,
    HnswParameters,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
    AzureOpenAIVectorizer,
    AzureOpenAIVectorizerParameters
)

In [3]:
import os
service_endpoint = os.getenv("AZSCH_ENDPOINT")  
credential = AzureKeyCredential(os.environ["AZSCH_KEY"])

print(service_endpoint)

api_key = os.getenv("AZURE_OPENAI_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")

https://ikaisch.search.windows.net


In [ ]:
# Create a search index
def create_search_index(index_name, model_name='text-embedding-ada-002'):
    index_client = SearchIndexClient(
        endpoint=service_endpoint, credential=credential)
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="proverb", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True,
                        analyzer_name="ko.microsoft"),
        SearchableField(name="meaning", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True,
                        analyzer_name="ko.microsoft"),
        SearchField(name="source", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True),
        SearchField(name="type", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True),
        SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                    searchable=True, vector_search_dimensions=1536 if model_name == 'text-embedding-ada-002' else 3072,
                    #stored=False,
                    vector_search_profile_name="myHnswProfile")  
    ]

    vector_search = VectorSearch(  
        algorithms=[  
            HnswAlgorithmConfiguration(  
                name="myHnsw",  
                parameters=HnswParameters(  
                    m=4,  
                    ef_construction=400,  
                    ef_search=500,  
                    metric=VectorSearchAlgorithmMetric.COSINE,  
                ),  
            )
        ],  
        profiles=[  
            VectorSearchProfile(  
                name="myHnswProfile",  
                algorithm_configuration_name="myHnsw",
                vectorizer_name="vectorizer"
            )
        ],
        vectorizers=[
            AzureOpenAIVectorizer(
                vectorizer_name="vectorizer",
                kind="azureOpenAI",
                parameters = AzureOpenAIVectorizerParameters(
                    resource_url=azure_endpoint,
                    api_key=api_key,
                    deployment_name=model_name,
                    model_name=model_name
                ),
            )
        ]
    )  

    semantic_config = SemanticConfiguration(  
        name="semantic-config",  
        prioritized_fields=SemanticPrioritizedFields(  
            title_field=SemanticField(field_name="proverb"),
            content_fields=[SemanticField(field_name="meaning")]  
        )
    )

    # Create the semantic search with the configuration  
    semantic_search = SemanticSearch(configurations=[semantic_config]) 

    # Create the search index
    index = SearchIndex(name=index_name, fields=fields,
                        vector_search=vector_search, semantic_search=semantic_search)
    result = index_client.create_or_update_index(index)
    print(f' {result.name} created')

def update_search_index_rank(index_name, include_title=False, model_name='text-embedding-3-large'):
    index_client = SearchIndexClient(
        endpoint=service_endpoint, credential=credential)
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="proverb", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True,
                        analyzer_name="ko.microsoft"),
        SearchableField(name="meaning", type=SearchFieldDataType.String,
                        searchable=True, retrievable=True,
                        analyzer_name="ko.microsoft"),
        SearchableField(name="source", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True),
        SearchableField(name="type", type=SearchFieldDataType.String,
                        searchable=False, retrievable=True),
        SearchField(name="vector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                    searchable=True, vector_search_dimensions=1536 if model_name == 'text-embedding-ada-002' else 3072,
                    vector_search_profile_name="myHnswProfile")  
    ]
    semantic_config = SemanticConfiguration(  
        name="semantic-config",  
        prioritized_fields=SemanticPrioritizedFields(  
            title_field=SemanticField(field_name="proverb") if include_title else None,
            content_fields=[SemanticField(field_name="meaning")]  
        )
    )

    # Create the semantic search with the configuration  
    semantic_search = SemanticSearch(configurations=[semantic_config]) 

    # Create the search index
    index = SearchIndex(name=index_name, fields=fields, semantic_search=semantic_search)
    result = index_client.create_or_update_index(index)
    print(f' {result.name} updated')

In [5]:
index_name = "korproverb-semantic-index"
#create_search_index(index_name, 'text-embedding-ada-002')
create_search_index(index_name, 'text-embedding-3-large')

 korproverb-semantic-index created


In [6]:
search_client = SearchClient(endpoint=service_endpoint, index_name=index_name, credential=credential)

In [7]:
import pandas as pd
from tqdm import tqdm

In [8]:
df = pd.read_pickle('kor_proverbs_embeddings.pkl')
df.head()

,Description,Meaning,Source,Embeddings2,Type,Embeddings3
0,가갸 뒤 자〔뒷다리〕도 모른다,"글자를 모르는 사람을 비유적으로 이르는 말.,사리에 어두운 사람을 놀림조로 이르는 말.",고려대 한국어대사전,"[0.005334234796464443, -0.014505433849990368, ...",속담,"[0.017118478193879128, -0.024258537217974663, ..."
1,가까운 남이 먼 일가보다 낫다,이웃끼리 서로 친하게 지내다 보면 먼 곳에 있는 일가보다 더 친하게 되어 서로 도우...,표준국어대사전,"[-0.007843011058866978, -0.018381033092737198,...",속담,"[-0.028828121721744537, -0.00209252187050879, ..."
2,가까운 데 집은 깎이고 먼 데 절은 비친다,가까운 데 있는 절이나 집은 자잘한 흠도 잘 드러나서 좋지 않아 보이고 먼 곳에 윤...,표준국어대사전,"[-0.0012391135096549988, -0.012439466081559658...",속담,"[-0.0034362655133008957, 0.012371719814836979,..."
3,가까운 무당보다 먼 데 무당이 영하다,흔히 사람은 자신이 잘 알고 가까이 있는 것보다는 잘 모르고 멀리 있는 것을 더 좋...,표준국어대사전,"[0.012985464185476303, -0.0004005177761428058,...",속담,"[0.023922793567180634, -0.011181957088410854, ..."
4,가까운 집 며느리일수록 흉이 많다,늘 가까이 있고 잘 아는 사이일수록 상대편의 결점이 눈에 더 많이 띈다는 말.,표준국어대사전,"[-0.0019731209613382816, -0.02421422116458416,...",속담,"[0.012853839434683323, -0.039608076214790344, ..."


In [16]:
count = 0
batch_size = 20
for i in tqdm(range(0, len(df), batch_size)):
    # set end position of batch
    i_end = min(i+batch_size, len(df))
    
    documents = df[i:i_end].apply(
        lambda row: {'id': str(row.name), 
                     'proverb': row['Description'], 
                     'meaning': row['Meaning'],
                     'source': row['Source'],
                     'type': str(row['Type']),
                     #'vector': generate_embeddings(row['chunk'])
                     'vector': row['Embeddings3']
                    }, axis=1).to_list()
    
    result = search_client.upload_documents(documents)  

100%|██████████| 224/224 [02:00<00:00,  1.86it/s]


### Vector Search

In [9]:
from azure.search.documents.models import (
    VectorizableTextQuery,
    VectorQuery,
    VectorizedQuery,
    QueryType,
    QueryCaptionType,
    QueryAnswerType)

In [10]:
def azsch_vector_query(query):
    #vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="vector", exhaustive=True)

    results = search_client.search(  
        search_text=None,  
        vector_queries=[vector_query],
        select=["proverb", "meaning", "source", "type"],
        query_language="ko-kr",
        top=10 # for limiting text search
    ) 

    for i, result in enumerate(results, 1): 
        print(f"{i}: {result['@search.score']:.10f}: {result['proverb']} - {result['meaning']}, {result['source']}")  

In [11]:
azsch_vector_query('노력')

1: 0.6639138000: 드나드는 개가 꿩을 문다 - 꾸준하게 열성적으로 노력하는 사람이 일을 이루고 재물을 얻을 수 있다는 말., 표준국어대사전
2: 0.6578187000: 십 년 적공이면 한 가지 성공을 한다 - 무슨 일이든지 오랫동안 꾸준히 노력하면 마침내는 성공하게 됨을 이르는 말., 우리말샘
3: 0.6566128000: 절뚝발이 말이 천리 간다 - 누구나 꾸준히 노력하면 무슨 일이든 이룰 수 있다는 말. (=둔한 말도 열흘 가면 천리를 간다.), 고려대 한국어대사전
4: 0.6485814000: 눈을 떠야 별을 보지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말., 표준국어대사전
5: 0.6459018000: 옥도 갈아야〔닦아야〕 빛이 난다 - 아무리 소질이 좋아도 그것을 잘 수양하지 않으면 훌륭해질 수 없음을 이르는 말.,고생을 겪으면서 노력해야 뜻한 바를 이룰 수 있다는 말., 고려대 한국어대사전
6: 0.6447277000: 내 침 발라 꼰 새끼가 제일 - 자기의 노력을 들여 이룩한 성과가 귀중함을 비유적으로 이르는 말., 표준국어대사전
7: 0.6434084000: 잠을 자야 꿈을 꾸지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말.,원인 없이 결과를 바랄 수 없음을 이르는 말., 표준국어대사전
8: 0.6410484000: 굴을 파야 금을 얻는다 - 목적을 이루기 위하여서는 거기에 필요한 조건을 갖추거나 노력을 하여야 함을 교훈적으로 이르는 말., 우리말샘
9: 0.6397308000: 정성이 지극하면 돌 위에도 풀이 난다 - 진심으로 정성을 다하면 불가능해 보이는 일도 이루어질 수 있다는 말., 고려대 한국어대사전
10: 0.6392494400: 쥐도 한몫 보면 락이 있다 - 한길로 전심전력하면 성공할 때가 있음을 비유적으로 이르는 말., 표준국어대사전


In [12]:
azsch_vector_query('노력 준비')

1: 0.6938119500: 눈을 떠야 별을 보지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말., 표준국어대사전
2: 0.6844479000: 새도 날려면 움츠린다 - 무슨 일이든 사전에 준비할 틈과 노력이 필요하다는 말. (=개구리도 옴쳐야 뛴다.), 고려대 한국어대사전
3: 0.6826436500: 한 달 싸우기 위해 팔 년 양병한다 - 무슨 일이나 확실한 성공을 거두기 위하여서는 준비에 많은 공을 들여야 한다는 말., 표준국어대사전
4: 0.6674417000: 거미도 줄을 쳐야 벌레를 잡는다 - 무슨 일이든지 거기 필요한 준비가 있어야 그 결과를 얻을 수 있다는 말., 표준국어대사전
5: 0.6660027500: 잠을 자야 꿈을 꾸지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말.,원인 없이 결과를 바랄 수 없음을 이르는 말., 표준국어대사전
6: 0.6623701000: 하늘을 보아야 별을 따지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말.,무슨 일이 이루어질 기회나 조건이 전혀 없음을 이르는 말., 표준국어대사전
7: 0.6496313000: 아직 신날도 안 꼬았다 - 큰일을 하려고 하면서도 조금도 준비가 되어 있지 않음을 비유적으로 이르는 말., 표준국어대사전
8: 0.6392748400: 이도 아니 나서 콩밥을 씹는다 - 아직 준비가 안 되고 능력도 없으면서 또는 절차를 넘어서 어려운 일을 하려고 달려듦을 비유적으로 이르는 말., 표준국어대사전
9: 0.6381285000: 개장수도 올가미가 있어야 한다 - 무슨 일을 하든지 거기에 필요한 준비와 도구가 있어야 함을 비유적으로 이르는 말., 표준국어대사전
10: 0.6364765000: 노루 본 놈이〔노루 보고〕 그물 짊어진다 - 무슨 일을 미리 준비하지 않고 일을 당해서야 허겁지겁 준비함을 비유적으로 이르는 말., 고려대 한국어대사전


In [13]:
azsch_vector_query('열심히 노력하면 성공한다')

1: 0.7491113000: 십 년 적공이면 한 가지 성공을 한다 - 무슨 일이든지 오랫동안 꾸준히 노력하면 마침내는 성공하게 됨을 이르는 말., 우리말샘
2: 0.7416658400: 드나드는 개가 꿩을 문다 - 꾸준하게 열성적으로 노력하는 사람이 일을 이루고 재물을 얻을 수 있다는 말., 표준국어대사전
3: 0.7394439000: 절뚝발이 말이 천리 간다 - 누구나 꾸준히 노력하면 무슨 일이든 이룰 수 있다는 말. (=둔한 말도 열흘 가면 천리를 간다.), 고려대 한국어대사전
4: 0.7362113000: 쥐도 한몫 보면 락이 있다 - 한길로 전심전력하면 성공할 때가 있음을 비유적으로 이르는 말., 표준국어대사전
5: 0.7297649000: 정성이 지극하면 돌 위에도 풀이 난다 - 진심으로 정성을 다하면 불가능해 보이는 일도 이루어질 수 있다는 말., 고려대 한국어대사전
6: 0.7077422000: 논 자취는 없어도 공부한 공은 남는다 - 놀지 않고 힘써 공부하면 훗날 그 공적이 반드시 드러난다는 말., 고려대 한국어대사전
7: 0.7058761000: 하자고 결심하면 못 해낼 일이 없다 - 결심과 각오만 단단하다면 무슨 일이든 성사할 수 있다는 말., 표준국어대사전
8: 0.7014453000: 옥도 갈아야〔닦아야〕 빛이 난다 - 아무리 소질이 좋아도 그것을 잘 수양하지 않으면 훌륭해질 수 없음을 이르는 말.,고생을 겪으면서 노력해야 뜻한 바를 이룰 수 있다는 말., 고려대 한국어대사전
9: 0.6961849300: 옥도 갈아야 빛이 난다 - 아무리 소질이 좋아도 이것을 잘 닦고 기르지 아니하면 훌륭한 것이 되지 못한다는 말.,고생을 겪으며 노력을 기울여야 뜻한 바를 이룰 수 있다는 말., 표준국어대사전
10: 0.6961849300: 옥석도 닦아야 빛이 난다 - 아무리 소질이 좋아도 이것을 잘 닦고 기르지 아니하면 훌륭한 것이 되지 못한다는 말.,고생을 겪으며 노력을 기울여야 뜻한 바를 이룰 수 있다는 말., 표준국어대사전


### Hybrid Search

In [14]:
def azsch_hybrid_query(query):
    #vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="vector", exhaustive=True)

    results = search_client.search(  
        search_text=query,  
        vector_queries=[vector_query],
        select=["proverb", "meaning", "source", "type"],
        query_language="ko-kr",
        top=10 # for limiting text search
    ) 

    for i, result in enumerate(results, 1): 
        print(f"{i}: {result['@search.score']:.10f}: {result['proverb']} - {result['meaning']}, {result['source']}")  

In [15]:
azsch_hybrid_query('열심히 노력하면 성공한다')

1: 0.0333333351: 십 년 적공이면 한 가지 성공을 한다 - 무슨 일이든지 오랫동안 꾸준히 노력하면 마침내는 성공하게 됨을 이르는 말., 우리말샘
2: 0.0312805474: 절뚝발이 말이 천리 간다 - 누구나 꾸준히 노력하면 무슨 일이든 이룰 수 있다는 말. (=둔한 말도 열흘 가면 천리를 간다.), 고려대 한국어대사전
3: 0.0307983905: 쥐도 한몫 보면 락이 있다 - 한길로 전심전력하면 성공할 때가 있음을 비유적으로 이르는 말., 표준국어대사전
4: 0.0293865316: 둔한 말이 열 수레를 끈다 - 미련하고 둔한 사람이라 하더라도 쉬지 않고 노력하면 큰일을 할 수 있다는 말., 고려대 한국어대사전
5: 0.0292139538: 누구는 날 때부터 안다더냐 - 사람이면 누구나 똑같으므로 열심히 노력하면 배울 수 있다는 말., 고려대 한국어대사전
6: 0.0288934410: 드나드는 개가 꿩을 문다 - 꾸준하게 열성적으로 노력하는 사람이 일을 이루고 재물을 얻을 수 있다는 말., 표준국어대사전
7: 0.0287784673: 눈을 떠야 별을 보지 - 어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 말., 표준국어대사전
8: 0.0277831368: 천 리 길을 찾아와서 문턱 넘어 죽는다 - 오랫동안 고생하며 추진하여 오던 일이 성공을 눈앞에 놓고 덜컥 잘못되는 경우를 비유적으로 이르는 말., 표준국어대사전
9: 0.0270125233: 소매가 길면 춤을 잘 추고 돈이 많으면 장사를 잘한다 - 수단이나 밑천이 든든하면 성공하기 쉽다는 말., 표준국어대사전
10: 0.0268759020: 진 꽃은 또 피지만 꺾인 꽃은 다시 피지 못한다 - 아무리 형편이 어렵더라도 뜻을 굳게 가지고 굽히지 아니하여야 끝내 성공할 수 있음을 비유적으로 이르는 말., 표준국어대사전


### Semantic rerank

combination of `title(description)` text + `meaning` text + `meaning` vector search result

In [16]:
def azsch_rerank_query(query):
    #vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
    vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="vector", exhaustive=True)

    results = search_client.search(  
        search_text=query,  
        vector_queries=[vector_query],
        select=["proverb", "meaning", "source", "type"],
        query_type=QueryType.SEMANTIC,
        semantic_configuration_name='semantic-config',
        query_caption=QueryCaptionType.EXTRACTIVE,
        #query_answer=QueryAnswerType.EXTRACTIVE,
        query_language="ko-kr",
        top=10 # for limiting text search
    ) 

    for result in results:  
        if result["@search.captions"]:
            caption = result["@search.captions"][0]
            print(f"{result['@search.reranker_score']:.5f}/{result['@search.score']:.5f}: {result['proverb']} - {caption.highlights}, {result['source']}")  
        else:
            print(f"{result['@search.reranker_score']:.5f}/{result['@search.score']:.5f}: {result['proverb']} - {result['meaning']}, {result['source']}")  
 

In [17]:
#update_search_index_rank(include_title=True)
azsch_rerank_query('열심히 노력하면 성공한다')

2.46223/0.02675: 구르는 돌은 이끼가 안 낀다 - <em>부지런하고 꾸준히 노력하는 사람은 침체되지 않고 계속 발전한다는 </em>말., 표준국어대사전
2.43476/0.02444: 가는 말에 채찍질 - <em>열심히 하고 있는데도 더 빨리하라고 독촉함을 비유적으로 이르는 말.</em>,형편이나 힘이 한창 좋을 때라도 더욱 마음을 써서 힘써야 함을 비유적으로 이르는 말., 표준국어대사전
2.41367/0.02037: 산에 가야 꿩을 잡고 바다엘 가야 고기를 잡는다 - <em>무엇인가를 이루기 위해서는 그 방향을 제대로 잡고 노력하여야만 그 목적을 제대로 이룰 수 있음을</em> 비유적으로 이르는 말.,무슨 일이든지 가만히 앉아 있어서는 이루어지지 않고 발 벗고 나서서 힘을 들여야 이루어짐을 비유적으로 이르는 말., 고려대 한국어대사전
2.40020/0.01915: 산에 가야 범을 잡는다 - <em>어떤 일을 이루기 위한 방향을 제대로 잡고 노력하여야만 그 일을 이룰 수 있음을</em> 비유적으로 이르는 말.,무슨 일을 하려면 발 벗고 나서서 실제로 힘들여 해야 함을 비유적으로 이르는 말. (=산에 가야 꿩., 고려대 한국어대사전
2.39488/0.02878: 눈을 떠야 별을 보지 - <em>어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 </em>말., 표준국어대사전
2.39187/0.03333: 십 년 적공이면 한 가지 성공을 한다 - <em>무슨 일이든지 오랫동안 꾸준히 노력하면 마침내는 성공하게 됨을 </em>이르는 말., 우리말샘
2.37964/0.02598: 내 침 발라 꼰 새끼가 제일 - <em>자기의 노력을 들여 이룩한 성과가 귀중함을 </em>비유적으로 이르는 말., 표준국어대사전
2.37698/0.02504: 잠을 자야 꿈을 꾸지 - <em>어떤 성과를 거두려면 그에 상당한 노력과 준비가 있어야 한다는 </em>말.,원인 없이 결과를 바랄 수 없음을 이르는 말., 표준국어대사전
2.37060/0.02485: 하늘을 보

#### Modify semantic configuration

In [18]:
update_search_index_rank(index_name, include_title=False)
azsch_rerank_query('열심히 노력하면 성공한다')

 korproverb-semantic-index updated
2.51575/0.02037: 산에 가야 꿩을 잡고 바다엘 가야 고기를 잡는다 - 무엇인가<em>를 이루기 위해서는 그 방향을 제대로 잡고 노력하여야만 그 목적을 제대로 이룰 수 있음을</em> 비유적으로 이르는 말.,무슨 일이든지 가만히 앉아 있어서는 이루어지지 않고 발 벗고 나서서 힘을 들여야 이루어짐을 비유적으로 이르는 말., 고려대 한국어대사전
2.47481/0.01136: 소같이 벌어서 쥐같이 먹어라 - 소같이<em> 꾸준</em><em>하고 힘써 일하여 많이 벌어서는 </em>쥐같이 조금<em>씩 먹으</em>라는 뜻으로,<em> 일은 열심히 </em>하여서 돈은 많이 벌고 생활은 아껴서 검소하게 하라는 말., 표준국어대사전
2.46595/0.03333: 십 년 적공이면 한 가지 성공을 한다 - 무슨 일이든지<em> 오랫동안 꾸준히 노력하면 마침내는 성공하게 됨을 </em>이르는 말., 우리말샘
2.45425/0.01333: 골나면 보리방아 더 잘 찧는다〔찧듯 한다〕 - 골나면 화가 치밀어 올라 그것을 해소하느라고 오히려<em> 일을 더 힘차고 열심히 하게 된다는</em> 말., 고려대 한국어대사전
2.43440/0.02921: 누구는 날 때부터 안다더냐 - 사람이면 누구나 똑같으므로<em> 열심히 노력하면 배울 수 </em>있다는 말., 고려대 한국어대사전
2.43369/0.01915: 산에 가야 범을 잡는다 - <em>어떤 일을 이루기 위한 방향을 제대로 잡고 노력하여야만 그 일을 이룰 수 있음을 비유적</em>으로 이르<em>는 </em>말.,무슨 일을 하려면 발 벗고 나서서 실제로 힘들여 해야 함을 비유적으로 이르는 말. (=산에 가야 꿩., 고려대 한국어대사전
2.41703/0.02444: 가는 말에 채찍질 - <em>열심히 하고 있는데도 더 빨리하라고 독촉함을 비유적으로 이르는 말.</em>,형편이나 힘이 한창 좋을 때라도 더욱 마음을 써서 힘써야 함을 비유적으로 이르는 말.,